https://arxiv.org/abs/1711.05772 / https://arxiv.org/abs/1802.04877

https://github.com/natashamjaques/magenta/blob/affective-reward/magenta/models/affective_reward/latent_gan.py

## Labeling data

In [1]:
#from pdb import set_trace
#set_trace()

In [2]:
import numpy as np

labels = np.empty([0, 2], dtype=int)
gens_keep = np.empty([0, ], dtype=object)
zs_keep = np.empty([0, 1, 128], dtype=float)

In [3]:
from pathlib import Path
vae_dir = Path('../../pytorch-text-vae') # https://github.com/iconix/pytorch-text-vae

n_samples = 100
temp = 0.2

%run -i $vae_dir/generate.py $vae_dir/reviews_sentences_state.pt reviews_sentences_stored_info.pkl --cache-path=$vae_dir/tmp --num-sample=$n_samples --temp=$temp

Namespace(cache_path='../../pytorch-text-vae/tmp', max_length=50, num_sample=100, print_z=False, saved_vae='../../pytorch-text-vae/reviews_sentences_state.pt', seed=None, stored_info='reviews_sentences_stored_info.pkl', temp=0.2, use_cuda=True)
Fetching cached info at ../../pytorch-text-vae/tmp/reviews_sentences_stored_info.pkl
Cache ../../pytorch-text-vae/tmp/reviews_sentences_stored_info.pkl loaded, total load time 0.6166176795959473
Found saved model ../../pytorch-text-vae/reviews_sentences_state.pt
MAX_SAMPLE: False; TRUNCATED_SAMPLE: True
Setting new random seed
---
UNK continues to UNK UNK at the part of the UNK manages to work at feat
---
artist is all at the kind of UNK who continues to create a live story
---
this by now and another version of the track when i wanted to make inspired by others
---
theres a bit of electronic music that he has a similar to the rest of what i far
---
talk about his new UNK UNK is taken in a few months ago
---
apart from the people who and serves 

---
in this UNK this this is the sense of time in which look at closed
---
in the because exactly what what is teaming up with no guitars and its next
---
in addition to UNK and it is a brand new remix which remix to be perfection


In [4]:
list(zip(range(len(gens)), gens)), zs

([(0,
   'UNK continues to UNK UNK at the part of the UNK manages to work at feat'),
  (1, 'artist is all at the kind of UNK who continues to create a live story'),
  (2,
   'this by now and another version of the track when i wanted to make inspired by others'),
  (3,
   'theres a bit of electronic music that he has a similar to the rest of what i far'),
  (4, 'talk about his new UNK UNK is taken in a few months ago'),
  (5, 'apart from the people who and serves as well as one of his next yet'),
  (6, 'with a part of UNK of UNK is the new single comes in music every feat'),
  (7,
   'artist are back in the this year and know how to make such a killer rework'),
  (8,
   'no reminds me of a tune in the week and give it into your light and soundtrack and enjoy'),
  (9,
   'speaking of the track though it would be taken from his creative recordings'),
  (10, 'as one of house stuck in this below and grab a one of tropical house'),
  (11,
   'to the cover of co produced by no surprise that 

In [5]:
#new_labels = np.array([(1, -1), (10, -1)])

# -1 is "bad" because it contains the banned word, 1 is "good"
banned = 'below'
labels = np.ones(n_samples, dtype=int)
labels[np.where(np.char.find(gens, banned) == -1)] = -1
zs_keep = zs

labels

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [6]:
#labels = np.concatenate((labels, new_labels))
#gens_keep = np.concatenate((gens_keep, np.array(gens)[np.array([i for i, l in new_labels])]))
#zs_keep = np.concatenate((zs_keep, np.array([z.cpu().numpy() for z in zs])[np.array([i for i, l in new_labels])]))

#gens_keep

In [7]:
#gens_keep.shape, zs_keep.shape, labels.shape

In [8]:
#np.get_printoptions()

In [9]:
#np.set_printoptions(threshold=np.inf)
#zs_keep

In [10]:
# -1 is "bad", 1 is "good"
#labels = np.ones(n_samples)

# for now, make up some random labels
#np.random.seed(42)
#ones = np.random.rand(n_samples) < 0.2
#labels[ones] = -1

#labels = np.array([l for i, l in labels], dtype=float)
#labels

In [11]:
from collections import Counter
c1 = Counter([word for gen in gens for word in gen.split()])
(banned, c1[banned]), c1.most_common(25)

(('below', 2),
 [('UNK', 104),
  ('the', 89),
  ('a', 70),
  ('of', 64),
  ('and', 64),
  ('to', 62),
  ('artist', 30),
  ('is', 30),
  ('with', 30),
  ('in', 28),
  ('up', 22),
  ('as', 20),
  ('it', 19),
  ('this', 18),
  ('has', 18),
  ('on', 17),
  ('for', 17),
  ('one', 16),
  ('track', 14),
  ('his', 14),
  ('at', 13),
  ('new', 13),
  ('from', 13),
  ('who', 12),
  ('remix', 12)])

## Data

In [12]:
batch_size = 16

In [13]:
from fastai.dataset import *

class LatentDataset(Dataset):
    def __init__(self, x, y): self.x,self.y = x,y
    def __getitem__(self, idx): return A(self.x[idx], self.y[idx])
    def __len__(self): return len(self.x)
    
ds = LatentDataset(zs_keep, labels.astype(float))
dl = DataLoader(ds, batch_size)
md = ModelData('.', dl, None)

In [14]:
md.trn_ds[0]

[array([[ 1.86998,  0.16694,  0.92986,  0.3011 ,  0.88631,  0.05123,  0.57243, -1.12455, -0.93751, -0.83214,
         -0.66173, -0.96187, -0.94249, -0.26238,  0.15395,  0.37429,  2.03871, -0.92495,  0.30149, -0.79014,
         -0.70165,  0.63867,  0.02045,  2.01053, -0.82896,  0.51366, -0.85344, -0.47532,  0.8978 , -0.41411,
          0.14668, -0.73412, -1.51044,  0.47573,  0.6505 , -1.29806,  0.1958 , -1.8802 ,  0.63649, -0.09794,
          1.96826, -0.13701,  1.18201, -0.53183,  0.71349,  1.12875,  1.37124, -0.98896,  1.4739 , -0.99211,
         -0.84446,  0.59717, -0.06869,  0.7448 , -1.53543,  2.15656,  0.70748,  1.33136,  0.04036,  1.4453 ,
         -1.44555, -0.60868,  2.07662,  1.85447, -2.30609, -0.97993,  0.24262, -0.59866, -0.41801,  0.07857,
         -1.4967 , -0.37301,  0.62523, -1.33673,  0.99978, -1.35683,  0.15687, -0.01955, -1.32605,  0.31812,
          0.3286 , -1.76496, -0.82619,  0.94327,  2.05381,  0.52204, -0.59554,  0.74836, -1.52927, -0.67017,
         -0.21197, 

## Model

In [15]:
n_latent = 128
n_hidden = 1024
lr = 3e-4

In [16]:
import torch.optim as optim
import torch.nn as nn

class LCGAN_D(nn.Module):
    '''Discriminator'''
    def __init__(self, n_latent, n_hidden=n_hidden, n_output=1):
        super(LCGAN_D, self).__init__()
        
        self.i2h = nn.Linear(n_latent, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        self.h2o = nn.Linear(n_hidden, n_output)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, z):
        x = z
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        v = self.sigmoid(self.h2o(x))
        
        return v

class LCGAN_G(nn.Module):
    '''Generator'''
    def __init__(self, n_latent, n_hidden=n_hidden):
        super(LCGAN_G, self).__init__()
        self.n_latent = n_latent
        
        self.i2h = nn.Linear(n_latent, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        # hidden-to-gating mechanism
        self.h2g = nn.Linear(n_hidden, 2*n_latent)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, z):
        x = z
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        x = self.h2g(x)
        
        # gating mechanism: allow network to remember/forget
        # what it wants to about the original z and x
        z_mid = x[:, self.n_latent:]
        gates = self.sigmoid(x[:, :self.n_latent])
        dz = gates * z_mid # TODO: why naming?
        z_prime = (1 - gates)*z + dz
        
        return z_prime

In [17]:
class LCGAN(nn.Module):
    def __init__(self, D, G, batch_size=batch_size):
        super(LCGAN, self).__init__()
        self.batch_size = batch_size
        
        self.D = D
        self.G = G

    def train(self):
        self.D.train()
        self.G.train()
        
    def forward(self, z=None):       
        if z is not None:
            # train discriminator
            v = self.D(z)
            return v
        else: # train GAN
            # gaussian random noise
            z_prior = torch.randn(self.batch_size, self.G.n_latent).cuda()
            
            z_prime = self.G(z_prior)
            v_prime = self.D(z_prime)
            
            return v_prime

## Training

In [18]:
gan = LCGAN(LCGAN_D(n_latent), LCGAN_G(n_latent)).cuda()

In [19]:
import fastai as fast

fast.core.set_trainable(gan.D, True)
fast.core.set_trainable(gan.G, True)

opt_d = optim.Adam(gan.D.parameters(), lr=lr)
opt_g = optim.Adam(gan.G.parameters(), lr=lr)

In [20]:
# test what the GAN is doing before any training

def generate(z):
    generated = vae.decoder.generate(z, 50, temp, True)
    generated_str = model.float_word_tensor_to_string(output_side, generated)

    EOS_str = f' {output_side.index_to_word(torch.LongTensor([EOS_token]))} '

    if generated_str.endswith(EOS_str):
        generated_str = generated_str[:-5]

    # flip it back
    return generated_str[::-1]
    
generate(gan.G(torch.randn(1, n_latent).cuda()))

'he listen to below and serves as one of the fact that its no times'

In [21]:
for i in range(10):
    print(generate(gan.G(torch.randn(1, n_latent).cuda())))

UNK is a listen to the below and rather from your feat
their their debut from album right now and and listen to the kind of r even though
is continues to long as part of the title song which has taken from previous today
artist is a listen to the rest of where he be taken from their ground
might have been looking for the release of UNK and UNK of trying to be taken from their previous times
in the UNK is back and serves as one in the the UNK or feat
has been a straight to the track and turns out of your soon as she feat
with the end of kanye UNK and UNK has a pretty trying to look at UNK feat
he been a listening to the UNK and serves as one of UNK my feat
UNK and artist has a straight out of the link week and theres no perfection


In [22]:
#from tqdm import tqdm

# adapted from: https://github.com/fastai/fastai/blob/master/courses/dl2/wgan.ipynb
def train(n_iter, first=False):
    gen_iters = 0
    for epoch in trange(n_iter):
        gan.train()
        data_iter = iter(md.trn_dl)
        i, n = 0, len(md.trn_dl)
        
        with tqdm(total=n) as pbar:
            while i < n:
                # train discriminator
                fast.core.set_trainable(gan.D, True)
                fast.core.set_trainable(gan.G, False)
                d_iters = 100 if (first and (gen_iters < 25) or (gen_iters % 500 == 0)) else 3
                j = 0
                
                while (j < d_iters) and (i < n):
                    j += 1; i += 1
                    batch = next(data_iter)
                    #print(j, i, batch[0].size(), batch[1].size())
                    z_real = batch[0].cuda()
                    v = gan(z_real)
                    
                    gan.D.zero_grad()
                    
                    loss_d = - (batch[1] * torch.log(v) + (1.0-batch[1]) * torch.log(1.0 - v)).mean()
                    loss_d.backward()
                    opt_d.step()
                    
                    pbar.update()
                
                # then train generator a little bit
                fast.core.set_trainable(gan.D, False)
                fast.core.set_trainable(gan.G, True)
                
                gan.G.zero_grad()
                
                #print(i, n)
                loss_g = - torch.log(gan()).mean()
                loss_g.backward()
                opt_g.step()
                gen_iters += 1
        
        print(f'Loss_D {to_np(loss_d)}; Loss_G {to_np(loss_g)}; ')

In [23]:
def train_and_generate(gan, n_epoch, n_sample=10):
    train(n_epoch)
    for i in range(n_sample):
        print(generate(gan.G(torch.randn(1, n_latent).cuda())))

In [24]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 246.00it/s]
Loss_D -3.0550546646118164; Loss_G 2.310281753540039; 
to aka artist is gearing up and one of the better known for times
artist has been trying to come along with a UNK and UNK and theres no feat
artist has with a one of the UNK and that sounds like every times
with new remix of UNK and we were gearing back to a a san where she feat
in the sort and serves as a vocals from trying to look at alone
the german producer artist has been trying to talk about soulful week and talking about feat
with a few months UNK and he listening to the original remix of UNK s or feat
artist who makes comes in the sort of UNK and trying to keep your playlist
continues to in UNK and the reminiscent of a track that would takes on your work
if UNK is a part of the track and single from white feat


In [25]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 283.49it/s]
Loss_D -18.071678161621094; Loss_G 10.242929458618164; 
he continues been gearing up for the original likes new UNK and theres few times
UNK feels like they featured on the full of trying to check it free feat
going on one on the UNK and artist continues to it into a hot times
as reminiscent of the UNK has a better known for every feat
part of the music comes along with a sort of trying to give it into your times
the electro the from artist were trying to put out of a look at feat
with this UNK and one and the trying to give it as a few times
along with a new remix of week and he comes into any perfection
in the remix of and i trying to take on a very good times
artist has been listening to the UNK and is one of your portland no feat


In [26]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 251.27it/s]
Loss_D -60.48027038574219; Loss_G 30.195158004760742; 
as a million recorded in the UNK UNK UNK to a better known as no feat
serves as a UNK a listening of UNK UNK where he feat
he listened to a UNK and part on the your UNK or feat
it seems to be one of the san where it it such a few times
later this UNK and artist has been listening to a soulful away from los feat
listen to serves as one of the music and were known for no feat
over a UNK and serves as one of the fact that youre look at times
UNK aka artist has featured on the likes the UNK and UNK or feat
artist has been trying to put back in the deep house what i cant feat
artist is back with the new version of the UNK and its no feat


In [27]:
g2 = []

for i in range(100):
    g2.append(generate(gan.G(torch.randn(1, n_latent).cuda())))
    
c2 = Counter([word for gen in g2 for word in gen.split()])
(banned, c2[banned]), c2.most_common(25)

(('below', 3),
 [('the', 97),
  ('of', 81),
  ('UNK', 75),
  ('a', 71),
  ('and', 71),
  ('to', 70),
  ('feat', 47),
  ('artist', 41),
  ('with', 36),
  ('from', 33),
  ('trying', 29),
  ('is', 26),
  ('times', 24),
  ('as', 24),
  ('has', 24),
  ('one', 23),
  ('new', 23),
  ('track', 22),
  ('it', 19),
  ('be', 18),
  ('remix', 16),
  ('in', 15),
  ('taken', 15),
  ('he', 12),
  ('look', 12)])

In [28]:
from pdb import set_trace
set_trace()

--Call--
> /home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/displayhook.py(247)__call__()
-> def __call__(self, result=None):
(Pdb) b ../../pytorch-text-vae/model.py:176
Breakpoint 1 at /home/ubuntu/pytorch-text-vae/model.py:176
(Pdb) c


In [29]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 220.42it/s]
Loss_D nan; Loss_G nan; 
> /home/ubuntu/pytorch-text-vae/model.py(176)sample()
-> top_i = torch.multinomial(output_dist, 1)[0]
(Pdb) output_dist
tensor([nan., nan., nan.,  ..., nan., nan., nan.], device='cuda:0')
(Pdb) output
tensor([[nan., nan., nan.,  ..., nan., nan., nan.]], device='cuda:0')
(Pdb) up
> /home/ubuntu/pytorch-text-vae/model.py(234)generate()
-> input, top_i = self.sample(output, temperature)
(Pdb) output
tensor([[nan., nan., nan.,  ..., nan., nan., nan.]], device='cuda:0')
(Pdb) i
0
(Pdb) z
tensor([[nan., nan., nan., nan., nan., nan., nan., nan., nan., nan.,
         nan., nan., nan., nan., nan., nan., nan., nan., nan., nan.,
         nan., nan., nan., nan., nan., nan., nan., nan., nan., nan.,
         nan., nan., nan., nan., nan., nan., nan., nan., nan., nan.,
         nan., nan., nan., nan., nan., nan., nan., nan., nan., nan.,
         nan., nan., nan., nan., nan., nan., nan., nan., nan., nan.,
         nan., nan., nan.,

BdbQuit: 

In [29]:
train_and_generate(gan, 1)

  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 7/7 [00:00<00:00, 276.98it/s]

Loss_D nan; Loss_G nan; 
100%|██████████| 1/1 [00:00<00:00, 34.44it/s]


RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1524586445097/work/aten/src/THC/generic/THCStorage.c:36

### Samples from other runs

In [28]:
train_and_generate(gan, 1)

  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00, 264.59it/s]

Loss_D 0.6728445291519165; Loss_G 2.7087178230285645; 
100%|██████████| 1/1 [00:00<00:00, 54.22it/s]
the link UNK and UNK is one of of trying to even more on many times
UNK is a better known of the UNK which is trying to look at my work
has a lot of trying to work in the UNK is make such more rather own times
where he has take and the one of the they are such a his times
you might as one of the take in there and a trying to look at times
artist hes teamed up with a dose of producer who has managed to keep very four again
in the remix of UNK and a is trying to take of some no times
UNK is a long as part of the star who has teamed up such a four style
artist has continues to come along with a sort of trying to make you your times
inspired by UNK and a listen to the based in san where we id feat


In [29]:
train_and_generate(gan, 1)

  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00, 269.69it/s]

Loss_D 0.558053731918335; Loss_G 2.9032042026519775; 
100%|██████████| 1/1 [00:00<00:00, 55.94it/s]
UNK is one of a indie rock where we we have trying to work
where they have a they take from UNK and such a very other vibes
one of l a four piece it will take on her first together
artist will be out on this UNK have and mash up on pop style
with a guitars and talk of the track which is trying to look at times
it will be back in the next ep and serves as a trying to stripped back in times
taken in the UNK and we have a gearing featured on our atlantic show
the UNK is a sort of especially where it come along with more sonic style
the last week the UNK is is trying to come along with their previous work
is one of the UNK and serves of a are trying to look at times


In [30]:
train_and_generate(gan, 1)

  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00, 264.33it/s]

Loss_D 0.471198707818985; Loss_G 2.8114919662475586; 
100%|██████████| 1/1 [00:00<00:00, 52.58it/s]
for a deep house where we have pretty trying to take on a house style
a new track is taken in the UNK and no way to better ready for their previous work
the london based producer artist has is managed to up with their previous work
artist are back in the sort of UNK and take on a way to inspired right again
according l a UNK and the party who is gearing up with some more style
inspired on on one of a UNK and it would is taken with their previous style
artist has been working on as one of a year where he managed to date
artist is one of the deep year where we have a listen to their previous work
in the london german producer artist has been trying to take on his previous work
UNK is a part of the track where they have teamed up with their own style


In [31]:
train_and_generate(gan, 1)

  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00, 271.77it/s]

Loss_D 0.399711012840271; Loss_G 2.113691806793213; 
100%|██████████| 1/1 [00:00<00:00, 54.35it/s]
we have managed to put it on a progressive house where to have a brand previous work
take of the week where we have even better known with their previous work
the london based producer who has it have a up with their previous work
and out of a year and we have gearing up with an electronic track
later last year and we have a gearing up with their previous work
it seems on one of the electro pop star there is gearing up a brand new tracks
the london based in and a person who a gearing up with their previous together
there is one of a progressive year where we have gearing up with a soulful together
of the last year and a been trying to take on their previous work
the out of a house where we have gearing up with their previous work


In [32]:
train_and_generate(gan, 1)

  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 4/4 [00:00<00:00, 264.64it/s]

Loss_D 0.3147360682487488; Loss_G 1.0290842056274414; 
100%|██████████| 1/1 [00:00<00:00, 53.97it/s]
there is a known for the next year and we are gearing up with new work music
for the next year and we have a gearing up with their previous work
we are gearing up l a progressive house where we are gearing up a new music
artist are gearing up for a a years ago and we a gearing up with their new work
it was a one of a indie label where we are gearing up to a more electronic music
there the one for a indie label where we trying to with a a previous work
artist are back in l a ago and we are gearing up with more electronic music
we have a gearing up in the and a person who a teamed up with some previous work
artist is gearing based in l a san where we have gearing up with electronic music
the london based in of a year where we have gearing up with their previous work


In [111]:
train_and_generate(gan, 1)

  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 201.62it/s]

Loss_D 0.6932061910629272; Loss_G 0.5500559210777283; 
100%|██████████| 1/1 [00:00<00:00, 75.18it/s]


' UNK artist artist check check out out UNK UNK UNK UNK UNK artist artist check UNK UNK artist artist check out UNK UNK UNK UNK artist artist check out UNK artist artist artist artist artist artist check out artist artist check out from artist song song song song you you'